In [ ]:
# Project 5: Railway Accidents
# Part 2: Clean data

In [ ]:
# Import pandas and re for cleaning the data
import pandas as pd
import re

# First we create a function we will need that splits the data up to extract numbers from string
def numb(Strings):
    newstr = ''.join((ch if ch in '0123456789' else ' ') for ch in i)
    i_num = [float(j) for j in newstr.split()]
    return i_num

# Read the CSV file and create a DataFrame from the csv file we created in previous code
df = pd.read_csv('https://raw.githubusercontent.com/Julardzija/Webscraping-Train-Accidents-Wiki/main/Datasets/WebscrapedTrainAccidents.csv', encoding = 'utf-8')

In [ ]:
# First we want to remove the entries that is either not a train accident or has no infobox
# This code makes sure that we remove all the rows that has these strings in infobox.
# ~ sign means that it will find the opposite.
df = df[~df.infobox.isin(["No infobox", "not a train accident"])]


# Reset the index of the DataFrame and drop the old index
# We do not neeed to remember the old index so we write True.
df = df.reset_index(drop=True)

# Drop unnecessary columns
df = df.drop(columns=["index"])
df = df.drop(columns=["infobox"])
df = df.drop(columns=["accident"])
df = df.drop(columns=["accident_url"])

In [ ]:
# Process and clean the 'Accident title' column
# We want to clean the accident titles as some of them have years included in them as
# beginning of the title, which does not fit well.
accident = []
for i in df["Accident title"]:
    i = i.lower()
    i = re.sub(r'[0-9]','', i)
    accident.append(i)
            


df["Accident title"] = accident
#Removes any white space at the beginning of a string
df["Accident title"] = df["Accident title"].str.lstrip()
#We have some titles where the year is important part of the name. For these we put the
#year of the accident back into the title.
df.loc[df["Accident title"].str.contains("crash of "), "Accident title"] = df["Accident title"] + df["year"].apply(str)
df.loc[df["Accident title"].str.contains("wreck of "), "Accident title"] = df["Accident title"] + df["year"].apply(str)
#Make each first letter in the words big
df['Accident title'] = df['Accident title'].str.title()

In [ ]:
# Process and clean the 'Location' column
# We have a lot of entries with footnotes (and normal brackets) inside of them. 
# We will make a code which we will reuse for many columns. But for now we will remove 
# square brakcet as well as normal brackets from Location.
# Then we want to further clean it. First we make all letters lowercase.
# We create a try except, to make sure the code can run throughout the data.
df['Location'] = df['Location'].str.lower()
location = []
for i in df["Location"]:
    try:
        i = re.sub(r"\[[^()]*\]", "", i)
        i = re.sub(r"\([^()]*\)", "", i)
        i = re.sub(r"2.4 miles east of ", "", i)
        i = re.sub(r"3 km south of ", "", i)
        i = re.sub(r"9 miles  east of", "", i)
        i = re.sub('[0-9]', '', i)
# This one removes a lot of coordinates. °.*$ means that whatever comes after the first symbol
# needs to be removed until the end of the string. The symbol included in removing.
        i = re.sub(r"°.*$", '', i)
        i = re.sub(r"coordinates.*$", '', i)
        i = re.sub(r"km.*$", '', i)
        
        #i = re.sub('km', '', i)
        location.append(i)
    except TypeError:
        location.append("")    
            
df["Location"] = location
# Now we want to split some of the data. There are many with too specific location followed by
# "near" a specific city. We want this as this might be more useful when trying to find 
# coordinates for this particular place.

# Split location values into two separate columns and clean up
df[["loc1", "loc2"]] = df["Location"].str.split("near ", expand=True)

df.loc[df.loc2.notnull(), "loc1"] = df.loc2
df["Location"] = df["loc1"]
df['Location'] = df['Location'].str.title()

# Drop temporary columns
df = df.drop(columns=["loc1", "loc2"])

In [ ]:
# Process and clean the 'Coordinates' column
# First we would want to separate coordinates into two separate columns latitude and longitude 
# which will help us for both cleaning & geomapping in the end. First we split between ";"
df[["Latitude", "Longitude"]] = df["Coordinates"].str.split(";", expand=True)

# The cleaning issue is now only with the Longitude column which sometimes has additional info
# This loop makes sure to clean it by removing any brackets and only giving us 
# numbers, dots and -   Similar code to location

Long=[]
for i in df["Longitude"]:
    try:
        i = re.sub(r"\([^()]*\)", "",i)
        i = re.sub(r"\[[^()]*\]", "",i)
        i = re.sub('[^0-9, ., -]', ' ', i)
        Long.append(i)
            
    except TypeError:
        Long.append("")

df["Longitude"] = Long


# Now that Coordinates are clean, we will put them together and have a clean version together.
df["Coordinates"] = df["Latitude"] + ';' + df["Longitude"]

# One of our tasks was to input the location if there was no coordinates in the data.
# This code is conditional. It locates any entry in coordinates column. The insna() function
# means we are looking for empty entries. If that is the case, then the row in coordinates
# will be replaced with the corresponding entry for location in same row.
df.loc[df.Coordinates.isna(), "Coordinates"] = df.Location

In [ ]:
# Process and clean the 'Cause' column
# For those that do not have any cause we want to write missing info
df.loc[df.Cause.isna(), "Cause"] = "Missing info"

# We need this column to behave similarly as we need to make a wordcloud. So we want it
# to be similar. We first remove square & normal bracket, as well as ' 
# Then we make all words small
# Lastly, we are interested in keeping three symbols: , ; and / 
# They separate different causes into the same entry, which we want to filtrate next.
cause = []
for i in df["Cause"]:
    try:
        i = i.lower()
        i = re.sub(r"\([^()]*\)", "",i)
        i = re.sub(r"\[[^()]*\]", "",i)
        i = re.sub('\'', '', i)
        i = re.sub('[^a-z, ;, \,, /, :]', '', i)
        cause.append(i)
            
    except AttributeError:
        cause.append("")

df["Cause"] = cause

# Lastly we have a lot of different causes for accidents. They are separated in the string
# with three signs seen below. We will split it and only keep the first cause string
# This was a decision made so we easier could categorize each accident to only single cause type.
df["Cause"] = df["Cause"].str.split(";", expand=True)[0]
df["Cause"] = df["Cause"].str.split(":", expand=True)[0]
df["Cause"] = df["Cause"].str.split("/", expand=True)[0]
df["Cause"] = df["Cause"].str.split("\,", expand=True)[0]


# Lastly we want to create a new column with categories for cause types. This is to be able
# to colorize them in the geomap when we visualize the final data.
# Human Error
df.loc[df["Cause"].str.contains("human"), "Causetype"] = "Human Error"
df.loc[df["Cause"].str.contains("driver"), "Causetype"] = "Human Error"
df.loc[df["Cause"].str.contains("engineer"), "Causetype"] = "Human Error"
df.loc[df["Cause"].str.contains("operator"), "Causetype"] = "Human Error"
df.loc[df["Cause"].str.contains("load"), "Causetype"] = "Human Error"
df.loc[df["Cause"].str.contains("weight"), "Causetype"] = "Human Error"
# Brake Failure/Error
df.loc[df["Cause"].str.contains("brak"), "Causetype"] = "Brake Failure/Error"
# Signalling Error
df.loc[df["Cause"].str.contains("signal"), "Causetype"] = "Signalling Error"
# Environmental Cause
df.loc[df["Cause"].str.contains("earthquake"), "Causetype"] = "Environmental Cause"
df.loc[df["Cause"].str.contains("wind"), "Causetype"] = "Environmental Cause"
df.loc[df["Cause"].str.contains("cloud"), "Causetype"] = "Environmental Cause"
df.loc[df["Cause"].str.contains("landsl"), "Causetype"] = "Environmental Cause"
# Track/Rail Error
df.loc[df["Cause"].str.contains("rail"), "Causetype"] = "Track/Rail Error"
df.loc[df["Cause"].str.contains("track"), "Causetype"] = "Track/Rail Error"
df.loc[df["Cause"].str.contains("line"), "Causetype"] = "Track/Rail Error"
# Derailment
df.loc[df["Cause"].str.contains("derail"), "Causetype"] = "Derailment"
# Under Investigation
df.loc[df["Cause"].str.contains("investigat"), "Causetype"] = "Under Investigation"
# Technical Error
df.loc[df["Cause"].str.contains("axl"), "Causetype"] = "Technical Error"
df.loc[df["Cause"].str.contains("wheel"), "Causetype"] = "Technical Error"
df.loc[df["Cause"].str.contains("joint"), "Causetype"] = "Technical Error"
df.loc[df["Cause"].str.contains("mainten"), "Causetype"] = "Technical Error"
df.loc[df["Cause"].str.contains("coupling rod"), "Causetype"] = "Technical Error"
df.loc[df["Cause"].str.contains("locomotive"), "Causetype"] = "Technical Error"
# Excessive Speed 
df.loc[df["Cause"].str.contains("speed"), "Causetype"] = "Excessive Speed"
# Bridge Failure 
df.loc[df["Cause"].str.contains("bridge"), "Causetype"] = "Bridge Failure"
# Cause information missing
df.loc[df["Cause"].str.contains("missing info"), "Causetype"] = "Cause information missing"
df.loc[df["Cause"].str.contains("undetermin"), "Causetype"] = "Cause information missing"
# Other
df.loc[df.Causetype.isna(), "Causetype"] = "Other"

In [ ]:
# Process and clean the 'Date' column
#Creating an empty list for the dates, when they have been cleaned.
Date_clean = []

# Creating a list of all the months, so we can search for them.
month = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

# Creating a for loop over all the date in the dataset.
for i in range(len(df["Date"])):

# Saving the year for the accident.
    year=df["year"][i]
    
# Removing all the date, that is Not a Number (NaN or empty)    
    if df["Date"][i]==df["Date"][i]:
        
# Making a string of the date.
        i=df["Date"][i]

# Cleaning the string from everthing, that is not a letter or number.
# Furthermore, we are removing all the [] and (), as well as what is written inside them.
        i=re.sub(r"\([^()]*\)", "",i)
        i=re.sub(r"\[[^()]*\]", "",i)
        i=re.sub("\xa0", " ",i)
        i=re.sub('[^a-z,0-9,A-Z, ]', '', i)
        
# Splitting the string up into a list, with all the remaning elements.
        i=i.split(" ")
        
# Creating a list to save the month we find.
        m=str()


# Finding and saving the month from the string.
        for j in month:
            if j in i:
                
                m=j
            
# Since the date of a day is maximum of two numbers, then we are removing everything
# that is longer the 2 elements.No other number than the date are left, since 
# all clocks and year are have 3 or more numbers.          
        for j in range(len(i)):
            i[j]=i[j].replace(",","")
            if len(i[j])>2:
                i[j]=""
        
        i=str(i)

# Creating a list, to save the clean date.
        date=[]

# If a date is left, we find the date, and save it.        
        if numb(i)!=[]:
            date.append(int(numb(i)[0]))
            
# Then we add the found month      
        date.append(m)
        
# And lastly we are adding the year.      
        date.append(year)     

# Then we are making it back to a string written as dd/mm/yyyy.
        date = str(date)[1:len(str(date))-1]
        date = date.replace("'","")

# Adding the clean date, to the list of clean dates.
        Date_clean.append(date)

# If the date was NaN, then we just add the year from df["year"], to the list.            
    else:
        Date_clean.append(year)

# Replace the old date with the clean date. 
df["Date"]=Date_clean



In [ ]:
# Cleaning up the data for Deaths as they are also messy


# We create an empty list we will use for sorting, like previous.
l=[]

for i in df["Deaths"]:
# Here we create a condition. If an entry is NAN then it is False. We are only interested in
# rows which have info in them.    
    if i==i:
# Removing all [],(), comma (,) and what are written inside them.
        i=i.lower()
        i=re.sub(r"\([^()]*\)", "",i)
        i=re.sub(r"\[[^()]*\]", "",i)
        i=re.sub("\,", "",i)
        
# Checking if there is only intergers (numbers).
        try: 
            int(i)
# If text is found, it will separate the number and string element with our function
        except ValueError:           
            i_num = numb(i)
            
# Splitting the remaning deaths into numberlist.
# If only one number in the list, then the number is set to number of deaths
            if len(i_num)==1:
                i=int(i_num[0])
            
# If there are two numbers, they shall be added together, under the following condition.
# We have looked through the deaths, and this is the cases, where the number needed to be added together. 
            elif i.find("plus ")>-1 or i.find("11 1 ")>-1 or i.find("passengers, ")>-1 or i.find(" and ")>-1:
            
                i=(int(i_num[0])+int(i_num[-1]))   
            
# Setting "none" to 0 deaths.
            elif i=="none":
                i=0
            
# All the remaning numbers are we taking the mean.
            else:           
                
                i=(int(i_num[0])+int(i_num[-1]))/2
            
# Making a list with all the number of deaths         
        l.append(int(i))
        
    else:
        l.append("Unknows number of deaths")
    
# Replacing the number of deaths with the list.
df["Deaths"]=l



In [ ]:
#CREATE CLEAN CSV DATA
# Now that we are finished with the cleaning process, we can export a csv file
# which we nearly can use for the final visualisation process
# We just need to add all the missing coordinats we can find, but we do that in another script.
df.to_csv('CleanData.csv', index=False, encoding='utf-8')


